In [1]:
import keras
from keras.layers import Input, Lambda, Dense, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import os, random
from PIL import Image
import h5py

/home/sree/anaconda3/envs/expmen/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
IMAGE_SIZE = [224, 224, 3]
epochs = 5
batch_size = 16

train_path = '/home/sree/testcnn/cnn'
valid_path = '/home/sree/testcnn/validation'
test_path = '/home/sree/testcnn/test'

train_files = glob(train_path + '/*/*.pn*g')
valid_files = glob(valid_path + '/*/*.pn*g')

classes = glob(train_path + '/*')
print('no of classes is :',len(classes))

In [ ]:
vgg = VGG16(input_shape=IMAGE_SIZE, weights='imagenet', include_top=False)

for layer in vgg.layers:
  layer.trainable = False

x = Flatten()(vgg.output)

prediction = Dense(len(classes), activation='softmax')(x)

model = Model(inputs=vgg.input, outputs=prediction)

model.summary()

keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'] )

In [ ]:
train_datagen = ImageDataGenerator(rotation_range=20, width_shift_range=0.1, height_shift_range=0.1, shear_range=0.1, zoom_range=0.2, horizontal_flip=False, vertical_flip=False )
valid_datagen = ImageDataGenerator(rescale = 1./255)
training_set = train_datagen.flow_from_directory(train_path, target_size = (224,224), batch_size = batch_size, class_mode = 'categorical')
validation_set = valid_datagen.flow_from_directory(valid_path, target_size = (224,224), batch_size = batch_size, class_mode = 'categorical')

In [ ]:
model.fit_generator(training_set,steps_per_epoch=5, epochs = epochs)

#model.save_weights('cnn_exp4_weights.h5') 

In [ ]:
# taking random image from validation path and tests CNN network accuracy
test_random = random.choice(os.listdir(test_path))
test_file=test_path +'//'+test_random
Image.open(test_file).show()
test_image = image.load_img(test_file, target_size = (224, 224))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)
#training_set.class_indices

In [ ]:
print(result)
result.argmax(axis=1)
a = result[0] # Can be of any shape
indices = np.where(a == a.max())
print(indices)
result[0]
max(result[0])